# Experiment 3: Poisoning Attacks in Federated Learning

* Poisoning of
    * experiment_id = 0; label flipping 5 -> 4,  poisoned_clients[0,10, 25, 50, 75, 100, 125, 150, 200] rounds 200

In [1]:
from federated_learning.utils import SHAPUtil
from federated_learning import ClientPlane, Configuration
from federated_learning.server import Server
from datetime import datetime
import random

In [2]:
class ObserverConfiguration():
    experiment_type = "shap_fl_poisoned"
    experiment_id = 0
    test = False
    dataset_type = "MNIST"
    
    # Client Configurations 
    client_name = "client"
    client_type = "client"
    
    # Server Configurations 
    server_name = "server"
    server_type = "server"
    server_id = 0

In [3]:
def set_rounds(rounds):
    client_plane.set_rounds(rounds)
    server.set_rounds(rounds)
    
def update_configs():
    client_plane.update_config(config, observer_config)
    server.update_config(config, observer_config)
    
def run_round(rounds):
    # Federated Learning Round 
    client_plane.update_clients(server.get_nn_parameters())
    selected_clients = server.select_clients()
    client_parameters = client_plane.train_selected_clients(selected_clients)
    server.aggregate_model(client_parameters)

def select_random_clean():
    idx = 0
    while idx in client_plane.poisoned_clients:
        idx = random.randint(0,config.NUMBER_OF_CLIENTS)
    return idx

def train_poisoned_client_only(rounds): 
    client_plane.clients[client_plane.poisoned_clients[0]].train(rounds)
    client_plane.clients[client_plane.poisoned_clients[0]].push_metrics()
    if rounds == 5: 
        print(client_plane.clients[client_plane.poisoned_clients[0]].train_dataloader.dataset.dataset.targets[client_plane.clients[client_plane.poisoned_clients[0]].poisoned_indices][0])
    
def train_clean_client_only(idx, rounds): 
    client_plane.clients[idx].train(rounds)
    client_plane.clients[idx].push_metrics()

In [4]:
config = Configuration()
config.FROM_LABEL = 5
config.TO_LABEL = 4
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader)
observer_config = ObserverConfiguration()

MNIST training data loaded.
MNIST test data loaded.


In [ ]:
#server.create_default_model()

In [5]:
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
server.create_default_model()
client_plane = ClientPlane(config, observer_config, data, shap_util)

default model saved to:./temp/models
Create 200 clients with dataset of size 300


In [ ]:
import numpy as np
for num_p_clients in [0, 10, 25, 50, 75, 100, 125, 150, 200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    client_plane.reset_poisoning_attack()
    config.POISONED_CLIENTS = num_p_clients
    update_configs()
    client_plane.poison_clients()
    clean_idx = select_random_clean()
    for i in range(200):
        set_rounds(i+1)
        run_round(i+1)
        if (i+1)%10 == 0:
            server.test()
            server.push_metrics()
        print("Round {} finished".format(i+1))

Load default model successfully
20/200 clients cleaned
40/200 clients cleaned
60/200 clients cleaned
80/200 clients cleaned
100/200 clients cleaned
120/200 clients cleaned
140/200 clients cleaned
160/200 clients cleaned
180/200 clients cleaned
200/200 clients cleaned
Cleaning successfully
No poisoning due to 0. poisoned clients
Round 1 finished
Round 2 finished
Round 3 finished
Round 4 finished
Round 5 finished
Round 6 finished
Round 7 finished
Round 8 finished
Round 9 finished

Test set: Average loss: 0.0011, Accuracy: 8277/10000 (83%)



Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.


Successfully pushed server data to victoria metrics
Round 10 finished
Round 11 finished
Round 12 finished
Round 13 finished
Round 14 finished
Round 15 finished
Round 16 finished
Round 17 finished
Round 18 finished
Round 19 finished

Test set: Average loss: 0.0006, Accuracy: 9019/10000 (90%)

Successfully pushed server data to victoria metrics
Round 20 finished
Round 21 finished
Round 22 finished
Round 23 finished
Round 24 finished
Round 25 finished
Round 26 finished
Round 27 finished
Round 28 finished
Round 29 finished

Test set: Average loss: 0.0004, Accuracy: 9263/10000 (93%)

Successfully pushed server data to victoria metrics
Round 30 finished
Round 31 finished
Round 32 finished
Round 33 finished
Round 34 finished
Round 35 finished
Round 36 finished
Round 37 finished
Round 38 finished
Round 39 finished

Test set: Average loss: 0.0003, Accuracy: 9413/10000 (94%)

Successfully pushed server data to victoria metrics
Round 40 finished
Round 41 finished
Round 42 finished
Round 43 finish

In [ ]:
import numpy as np
import torch
torch.sum(server.confusion_matrix)

In [ ]:
server.get_shap_values()